In [2]:
#Get help here: https://dynamic-fba.readthedocs.io/en/latest/examples.html

#Important! This code needs to be run in a different environment! 
# On Trine's computer, the environment is called dfba

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort./ 
                                                                               ]/ failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - dfba -> python[version='>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.8,<3.9.0a0']

Your python: python==3.9.7=h12debd9_1

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained 

In [5]:
from dfba import DfbaModel, ExchangeFlux, KineticVariable
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model

## 1. Generate dfba model

In [8]:
infilename = '../../models/iMM904_progesterone_fixed.xml'
print(f"Loading {infilename}")

fba_model = read_sbml_model(infilename)
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)

Loading ../../models/iMM904_progesterone_fixed.xml


## 2. Kinetic variables

 - The metabolites *j* in the medium will be XX
 - Later we will establish kinetic rules for the uptakes of these metabolites. 
 - In addition, we need to set a variable to keep track of biomass.
 - Instantiate kinetic variables to appear in model. The last command adds the kinetic variables to the model. 

In [10]:
#Does these variables have to have the same name as in the model???

X = KineticVariable("Biomass")
Gluc = KineticVariable("Glucose")
Xyl = KineticVariable("Xylose")
Oxy = KineticVariable("Oxygen")
Eth = KineticVariable("Ethanol")

dfba_model.add_kinetic_variables([X, Gluc, Xyl, Oxy, Eth])

In [23]:
X

yval[0]

In [16]:
print(dfba_model.kinetic_variables) #what does this mean.

[yval[0], yval[1], yval[2], yval[3], yval[4]]


## 3. Exchange fluxes

Instantiate exchange fluxes to appear in model, with ids corresponding to exchange reactions of the cobrapy model. 

The last command adds the exchange fluxes to the model

In [21]:
mu = ExchangeFlux("BIOMASS_SC5_notrace")  #IS THIS THE CORRECT BIOMASS???
v_G = ExchangeFlux("EX_glc__D_e")
v_Z = ExchangeFlux("EX_xyl__D_e")
v_O = ExchangeFlux("EX_o2_e")
v_E = ExchangeFlux("EX_etoh_e")

#model.reactions.EX_etoh_e

dfba_model.add_exchange_fluxes([mu, v_G, v_Z, v_O, v_E])

Ignoring exchange flux BIOMASS_SC5_notrace since italready exists in the model.
Ignoring exchange flux EX_glc__D_e since italready exists in the model.
Ignoring exchange flux EX_xyl__D_e since italready exists in the model.
Ignoring exchange flux EX_o2_e since italready exists in the model.
Ignoring exchange flux EX_etoh_e since italready exists in the model.


## 4. Rhs expressions
Provide symbolic expression for calculating the time derivtive of each kinetic variable currently in the model. 

See how these correspond to our ODE system:

\begin{align}
    \frac{dX}{dt} &= v_{BM}X \\
    \frac{dC_j}{dt} &= v_jX
\end{align}

In [ ]:
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559/1000 * X) # v_G [mmol gDW^-1 h^-1] * 0.18 g/mmol * gDW/L
dfba_model.add_rhs_expression("Xylose", v_Z * 150.13/1000 * X)
dfba_model.add_rhs_expression("Oxygen", 0) # O2 is kept constant
dfba_model.add_rhs_expression("Ethanol", v_E * 46.06844/1000 * X)